## Eval Models

In [107]:
import sys
sys.path.append('../src') 
sys.path.append('../') 
from utils.utils import Model, Trainer, ClsModel, RgrModel
from data import CIFAR10DLGetter, CIFAR10Extended
from models import HighDimModel, CategoricalModel, BaselineModel, W2VLabelModel
from src import load_config_file, load_datasets, config_model
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, random_split
import os
from datetime import datetime
from pathlib import Path
import argparse
import json
import torch.nn.functional as F

In [108]:
def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

# restores the tensors to their original scale
def denorm(batch, mean, std):
    """
    Convert a batch of tensors to their original scale.

    Args:
        batch (torch.Tensor): Batch of normalized tensors.
        mean (torch.Tensor or list): Mean used for normalization.
        std (torch.Tensor or list): Standard deviation used for normalization.

    Returns:
        torch.Tensor: batch of tensors without normalization applied to them.
    """
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    return batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1)

def high_dim_pass(model, device, data, epsilon):
    model.network.zero_grad()
    inputs, high_dim_labels, cat_labels = data
    inputs, high_dim_labels, cat_labels = inputs.to(device), \
                                              high_dim_labels.to(device), \
                                              cat_labels.to(device)
    # Set requires_grad attribute of tensor. Important for Attack
    inputs.requires_grad = True
    # Forward pass the data through the model
    yhat_regr = model.forward((inputs))
    yhat_cls = model.classify_predictions(yhat_regr)
    
    # Calculate the loss
    loss = F.huber_loss(yhat_regr, high_dim_labels)
    # Zero all existing gradients
    # model.network(zero_grad())
    # Calculate gradients of model in backward pass
    loss.backward()
    # Collect ``datagrad``
    data_grad = inputs.grad.data
    # Restore the data to its original scale
    data_denorm = denorm(inputs,  mean = [0.4914, 0.4822, 0.4465], std = [0.247, 0.243, 0.261])
    # Call FGSM Attack
    perturbed_data = fgsm_attack(data_denorm, epsilon, data_grad)

    # Reapply normalization
    perturbed_data_normalized = transforms.Normalize([0.4914, 0.4822, 0.4465],
                                                    [0.247, 0.243, 0.261])(perturbed_data)

    # Re-classify the perturbed image
    yhat_rgr_adv = model.forward((perturbed_data_normalized))

    # Check for success
    final_preds = model.classify_predictions(yhat_rgr_adv)
    num_correct, num_incorrect = model.calc_cls_accuracy(final_preds, cat_labels) 
    return num_correct, num_incorrect, final_preds

def categorical_pass(model, device, data, epsilon):
    model.network.zero_grad()
    inputs, labels = data
    inputs, labels= inputs.to(self.device), labels.to(self.device)
    inputs.requires_grad = True
    logits = self.forward(inputs)
    preds = torch.argmax(logits, dim=1)
    loss = F.nll_loss(logits, labels)
    loss.backward()
    
    # Collect ``datagrad``
    data_grad = inputs.grad.data
    # Restore the data to its original scale
    data_denorm = denorm(inputs,  mean = [0.4914, 0.4822, 0.4465], std = [0.247, 0.243, 0.261])
    # Call FGSM Attack
    perturbed_data = fgsm_attack(data_denorm, epsilon, data_grad)

    # Reapply normalization
    perturbed_data_normalized = transforms.Normalize([0.4914, 0.4822, 0.4465],
                                                    [0.247, 0.243, 0.261])(perturbed_data)
    yhat_rgr_adv = model.forward((perturbed_data_normalized))
    final_preds = torch.argmax(logits, dim=1)
    num_correct, num_incorrect = model.calc_cls_accuracy(final_preds, labels) 
    return final_acc, adv_examples, final_preds

In [109]:
def test(model, model_type, device, test_loader, epsilon):

    # Accuracy counter
    total_correct = 0
    total_incorrect = 0
    adv_predictions = []
    adv_examples = []

    # Loop over all examples in test set
    for _, data in enumerate(test_loader, 0):
        if model_type == "high_dim":
            num_correct, num_incorrect, final_preds = high_dim_pass(model, device, data, epsilon)
        else:
            num_correct, num_incorrect, final_preds = categorical_pass(model, device, data, epsilon)
        total_correct += num_correct
        total_incorrect += num_incorrect
        adv_predictions.append(final_preds)

    # Calculate final accuracy for this epsilon
    final_acc = total_correct/(total_incorrect + total_correct)
    print(f"Epsilon: {epsilon}\tTest Accuracy = {total_correct} / {total_incorrect + total_correct} = {final_acc}")

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples

## Categorical 

In [106]:
model_config_path = "../config/models/categorical/vgg11_categorical.json"
run_config_path = "../config/data_constrained_runs/local_run_params.json"
weights = ["../data/models/categorical/categorical_model_1", 
           "../data/models/categorical/categorical_model_2", 
           "../data/models/categorical/categorical_model_3"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_config = load_config_file(model_config_path)
run_config = load_config_file(run_config_path)
train_pct = run_config["train_pct"][0]
val_pct = run_config["val_pct"]

train_loader, val_loader, test_loader, high_dim_label_set = load_datasets(run_config, model_config, train_pct)
model, batch_size = config_model(model_config, high_dim_label_set)
for weight in weights: 
    model.network.load_state_dict(torch.load(weight, map_location=torch.device('cpu')))
    _, accuracy = model.eval_performance(test_loader)
    print(accuracy)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
<bound method Module.children of BaselineModel(
  (base_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1),

In [105]:
model

In [ ]:
accuracies = []
examples = []
mean = [0.4914, 0.4822, 0.4465]
std = [0.247, 0.243, 0.261]
# Run test for each epsilon
epsilons = [0.05, 0.1, 0.15, 0.2, .25, 0.3]
for eps in epsilons:
    acc, ex = test(model, "high_dim", device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

## Speech

In [100]:
model_config_path = "../config/models/high_dim/vgg11_speech.json"
run_config_path = "../config/data_constrained_runs/local_run_params.json"
weights = ["../data/models/speech/speech_model_1",
           "../data/models/speech/speech_model_2", 
           "../data/models/speech/speech_model_3"]

In [101]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_config = load_config_file(model_config_path)
run_config = load_config_file(run_config_path)
train_pct = run_config["train_pct"][0]
val_pct = run_config["val_pct"]

train_loader, val_loader, test_loader, high_dim_label_set = load_datasets(run_config, model_config, train_pct)
model, batch_size = config_model(model_config, high_dim_label_set)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [102]:
for weight in weights: 
    model.network.load_state_dict(torch.load(weight, map_location=torch.device('cpu')))
    _, accuracy = model.eval_performance(test_loader)
    print(accuracy)

Evaluating


KeyboardInterrupt: 

In [103]:
accuracies = []
examples = []
mean = [0.4914, 0.4822, 0.4465]
std = [0.247, 0.243, 0.261]
# Run test for each epsilon
epsilons = [0.05, 0.1, 0.15, 0.2, .25, 0.3]
for eps in epsilons:
    acc, ex = test(model, "high_dim", device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0.05	Test Accuracy = 2888 / 10000 = 0.2888
Epsilon: 0.1	Test Accuracy = 1534 / 10000 = 0.1534
Epsilon: 0.15	Test Accuracy = 1160 / 10000 = 0.116
Epsilon: 0.2	Test Accuracy = 1105 / 10000 = 0.1105
Epsilon: 0.25	Test Accuracy = 1127 / 10000 = 0.1127
Epsilon: 0.3	Test Accuracy = 1192 / 10000 = 0.1192


## W2V

In [23]:
model_config_path = "../config/models/high_dim/vgg11_w2v.json"
run_config_path = "../config/data_constrained_runs/local_run_params.json"
weights = ["../data/models/w2v/w2v_model_1", "../data/models/w2v/w2v_model_2", "../data/models/w2v/w2v_model_3"]

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_config = load_config_file(model_config_path)
run_config = load_config_file(run_config_path)
train_pct = run_config["train_pct"][0]
val_pct = run_config["val_pct"]

train_loader, val_loader, test_loader, high_dim_label_set = load_datasets(run_config, model_config, train_pct)
model, batch_size = config_model(model_config, high_dim_label_set)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [27]:
for weight in weights: 
    model.network.load_state_dict(torch.load(weight, map_location=torch.device('cpu')))
    _, accuracy = model.eval_performance(test_loader)
    print(accuracy)

Evaluating
0.8176
Evaluating
0.8169
Evaluating
0.8155


In [80]:
accuracies = []
examples = []
mean = [0.4914, 0.4822, 0.4465]
std = [0.247, 0.243, 0.261]
# Run test for each epsilon
epsilons = [0.05, 0.1, 0.15, 0.2, .25, 0.3]
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)



Epsilon: 0.05	Test Accuracy = 2909 / 10000 = 0.2909
Epsilon: 0.1	Test Accuracy = 2464 / 10000 = 0.2464
Epsilon: 0.15	Test Accuracy = 2120 / 10000 = 0.212
Epsilon: 0.2	Test Accuracy = 1741 / 10000 = 0.1741
Epsilon: 0.25	Test Accuracy = 1530 / 10000 = 0.153
Epsilon: 0.3	Test Accuracy = 1366 / 10000 = 0.1366


In [76]:
accuracies

[]